### ESCI 5331 Assignment 5

Jenny Huang 
### Description of file inputs / outputs
*Assumes following:* 


For ba6 file:
 - 1D vertical profile domain
 - Top boundary cond constant head
 - Bottom boundary cond constant head
 - Initial head: solve for hyd gradient consistent with Q, K, and h_Bot
 - constant DZ, DELR, DELC discretization 

For dis file:
 - NPER = 1 ITMUNI = 4 (day) LENUNI = 2 (m)
 - LAYCBD = 0 for all layers (i.e. no quasi-3D confining bed under layer)

Hard-coded file names in .nam file:
 - test.lst     (MODFLOW output file, summary output)
 - test.pcg (MODFLOW input file for solver)
 - test.oc  (MODFLOW input file for output specifications)
 - test.bud     (MODFLOW output file)
 - testhead.dat (MODFLOW output file)
 - ibound.dat (MODFLOW output file)
 - [testrech.bud (MODFLOW output file)]

(Note: For arrays, must have values for all columns on same line!  This is not req'd for PHT3D arrays....)


### Code to write input functions

In [1]:
import os
import numpy as np
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [2]:
def write_modflow_input_files(input_dir, **params):
    '''I just refactored the existing code to be a function 
    where input parameters can be passed to adjust the initial conditions 
    and parameters of the model. '''
    MODtest_dir = input_dir #dictionary where input files are stored
    MOD_dir = 'C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs'
    os.makedirs('C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs', exist_ok = True) #ensure dictionary exists
    slashstr = '/'
    #set parameters to passed values or default values 
    #model geometry
    nlay = params.get('nlay', 50)
    ncol = params.get('ncol', 1)
    domain_len = params.get('domain_len', 0.3)
    domain_bot_elev = params.get('domain_bot_elev', -1)
    domain_top_elev = params.get('domain_top_elev', 0)
    Z = (domain_top_elev - domain_bot_elev)
    dz = Z/nlay

    #head
    TopHead = params.get('TopHead', 1.5)
    vertgrad = params.get('vertgrad', -0.0038) # dh/dz, <0 for upward flux
    fl_recharge = params.get('fl_recharge', 0)
    rch_rate= params.get('rch_rate', 5e-4)

    #K
    Ks = params.get('Ks', {'0' : (6, 50)}) #a dictionary of layer number and (K, thickness) where 0 = top and increases going down
    layer_averaged_K = params.get('layer_averaged_K', False) #if True, compute averaged K given the K 
    hydcond = np.ones((nlay, ncol))  
    def calculate_avg_K(Ks, dz):#Ks = directory of conductivity layers as described above, dz = cell length
        layer_start = 0
        total_thickness = 0
        L_K = 0
        for layer, (K, thickness) in Ks.items():
            layer_thickness = thickness * dz
            L_K += layer_thickness/K
            total_thickness += layer_thickness 
        K_avg = total_thickness/L_K
        return K_avg
    if layer_averaged_K:
        Kavg = calculate_avg_K(Ks, dz)
        Ks = {'0': (Kavg, nlay)}

    #set hydcond values according to Ks 
    layer_start = 0
    for layer, (K, thickness) in Ks.items():
        layer_end = layer_start + thickness 
        hydcond[layer_start:layer_end] =K
        layer_start = layer_end 

    # -- everything below this point is the same as the original script! 
    # -- names of input files (created with this script)
    fil_ba6 = 'test_1D.ba6'
    fil_dis = 'test_1D.dis'
    fil_lpf = 'test_1D.lpf'
    fil_nam = 'test.nam'
    fil_lmt = 'test.lmt'

    # the following always have pre-set entries
    fil_pcg = 'test.pcg'
    fil_oc = 'test.oc'

    # the following is created only if recharge stress package is used
    if (fl_recharge):
        fil_rch = 'test.rch'   
    # MODtest_dir0 = os.path.join(MOD_dir, MODtest_dir)
    MODtest_dir0 = MOD_dir + slashstr + MODtest_dir

    if not (os.path.exists(MODtest_dir0)):
        os.makedirs(MODtest_dir0)

    # LPF parameters
    flow_filunit = 34  # 0: no cell-by-cell flow terms written, 
                        # >0: cell-by-cell flow terms written to this unit number when "SAVE BUDGET", 
                        # <0: cell-by-cell flow for constant-head cells in listing file when "SAVE BUDGET"
                        
    # - discretization
    DELR = domain_len / ncol  
    DELC = DELR

    IBOUND = np.ones((nlay, ncol))  # 1: variable head
    IBOUND[0,:] = -1  # -1: const head at top 
    IBOUND[-1,:] = -1  # -1: const head at bottom
    initHead = TopHead + vertgrad*dz*np.arange(0, -nlay, -1).reshape(nlay, -1)
    initHead = np.matlib.repmat(initHead, 1, ncol)

    if ((fl_recharge) and np.size(np.where(IBOUND[0,:] < 0) > np.array(0))):
        print('Error! Top layer must be variable head (IBOUND > 0) to receive recharge! Exiting...\n')


    # get layer elevations
    botm = np.linspace(domain_bot_elev, domain_top_elev, nlay+1).reshape(nlay+1, -1)
    botm = botm[-2::-1]
    top_sys = domain_top_elev  # if head > top, this is confined condition


    # -- pcg and oc files are not changed with this script
    fil_pcg_0 = MODtest_dir0 + slashstr + fil_pcg
    fid = open(fil_pcg_0, 'wt')
    fid.write("# Preconditioned conjugate-gradient package\n")
    fid.write("        50        30         1      MXITER, ITER1, NPCOND\n")
    fid.write("  0000.001      .001        1.         2         1         1      1.00\n")
    fid.write(" HCLOSE,      RCLOSE,    RELAX,    NBPOL,     IPRPCG,   MUTPCG    damp\n")
    fid.close()

    fil_oc_0 = MODtest_dir0 + slashstr + fil_oc
    fid = open(fil_oc_0, 'wt')
    fid.write("HEAD PRINT FORMAT 20\n")
    fid.write("HEAD SAVE UNIT 51\n")
    fid.write("COMPACT BUDGET AUX\n")
    fid.write("IBOUND SAVE UNIT 52\n")
    fid.write("PERIOD 1 STEP 1\n")
    fid.write("   PRINT HEAD\n")
    fid.write("   SAVE HEAD\n")
    fid.write("   PRINT BUDGET\n")
    fid.write("   SAVE BUDGET\n")
    fid.write("   SAVE IBOUND\n")
    fid.close()

    #-- ba6 file
    fil_ba6_0 = MODtest_dir0 + slashstr + fil_ba6


    fid = open(fil_ba6_0, 'wt')
    fid.write("# basic package file --- x-section %d layers, %d column\n" % (nlay, ncol))
    fid.write("XSECTION\n")
    fid.write("INTERNAL          1 (FREE)  3\n")
    for i in range(nlay):
        for j in range(ncol):
            fid.write("%4d " % np.transpose(IBOUND)[j, i])
        fid.write("\n")
    fid.write("    999.99  HNOFLO\n")
    fid.write("INTERNAL          1.0 (FREE)  3\n")
    for i in range(nlay):
        for j in range(ncol):
            fid.write("%7g " % np.transpose(initHead)[j, i])
        fid.write("\n")
    fid.close()

    # -- dis file
    # for now, assume these inputs:
    nrow = 1  # 1 for x-section
    nper = 1  # number stress periods
    itmuni = 4  # time units, 4: day
    lenuni = 2  # length units, 2: m
    LAYCBD = np.zeros((1, nlay))  # 0: no quasi-3D confining bed under layer
    perlen = 1  # length of period (doesn't matter for steady-state)
    nstp = 1  # number of time steps (doesn't matter for steady-state)
    tsmult = 1  # multiplier for length of successive time steps
    SsTr_flag = 'SS'  # 'SS' for steady-state, 'Tr' for transient

    fil_dis_0 = MODtest_dir0 + slashstr + fil_dis


    fid = open(fil_dis_0, 'wt')
    fid.write(" %d  %d  %d  %d  %d  %d " % (nlay, nrow, ncol, nper, itmuni, lenuni))
    fid.write("  NLAY,NROW,NCOL,NPER,ITMUNI,LENUNI\n")
    for i in range(nlay):
        fid.write(" %d" % LAYCBD[0, i])
    fid.write("\n")

    fid.write("CONSTANT %14g   DELR\n" % DELR)
    fid.write("CONSTANT %14g   DELC\n" % DELC)
    fid.write("CONSTANT %14g   TOP of system\n" % top_sys)
    for ii in range(1, nlay+1):
        a = botm[ii-1]
        fid.write("CONSTANT %14g   Layer BOTM layer %4d\n" % (a[0], ii))
    for ii in range(1, nper+1):
        fid.write(" %g %d %g %s        PERLEN, NSTP, TSMULT, Ss/Tr (stress period %4d)\n" % (perlen, nstp, tsmult, SsTr_flag, ii))
    fid.close()


    # -- top of lpf file (use Ksat_realization or other to create Ksat inputs)
    # assumed input values
    hdry = 1e30  # head assigned to dry cells
    nplpf = 0    # number of LPF parameters (if >0, key words would follow)
    laytyp = np.zeros((nlay, 1)) 
    laytyp[0, 0] = 1  # top is "covertible", rest are "confined"
    layave = np.zeros((nlay, 1))  # harmonic mean for interblock transmissivity
    chani = np.ones((nlay, 1))   # constant horiz anisotropy mult factor (for each layer)
    layvka = np.zeros((nlay, 1))  # flag 0: vka is vert K >0 is vertK/horK ratio
    laywet = np.zeros((nlay, 1))  # flag 0: wetting off



    fil_lpf_0 = MODtest_dir0 + slashstr +  fil_lpf
    fid = open(fil_lpf_0, 'wt')
    fid.write("# LPF package inputs\n")
    fid.write("%d %g %d    ILPFCB,HDRY,NPLPF\n" % (flow_filunit, hdry, nplpf))
    for i in range(nlay):
        fid.write("%2d" % laytyp[i, 0])
    fid.write("     LAYTYP\n")
    for i in range(nlay):
        fid.write("%2d" % layave[i, 0])
    fid.write("     LAYAVE\n")
    for i in range(nlay):
        fid.write("%2d" % chani[i, 0])
    fid.write("     CHANI\n")
    for i in range(nlay):
        fid.write("%2d" % layvka[i, 0])
    fid.write("     LAYVKA\n")
    for i in range(nlay):
        fid.write("%2d" % laywet[i, 0])
    fid.write("     LAYWET\n")


    # -- Write HKSAT in .lpf file
    for lay in range(1, nlay+1):
        fid.write("INTERNAL   1.000E-00 (FREE)    0            HY layer  %d\n" % (lay))
        for j in range(ncol):    
            fid.write(" %4.1f" % hydcond[lay-1, j])
        fid.write(" \n")

        fid.write("INTERNAL   1.000E-00 (FREE)    0            VKA layer  %d\n" % (lay))
        for j in range(ncol):    
            fid.write(" %4.1f" % hydcond[lay-1, j])
        fid.write(" \n")
    fid.close()


    # -- .rch file
    if (fl_recharge):
        NRCHOP = 3  # option 3: apply recharge rate to highest active cell in column
        IRCHBD = -1 # <0 to skip writing cell-by-cell flow terms to file
        INRECH = 1 # >0: read in recharge values below (<0: use recharge from previous stress period)
        fil_rch_0 = MODtest_dir0 + slashstr + fil_rch
        fid = open(fil_rch_0, 'wt')
        fid.write('# recharge package \n')  
        fid.write('    %2d    %2d        NRCHOP,IRCHBD\n' % (NRCHOP, IRCHBD)) 
        for per_i in range(1, nper+1):
            fid.write('    %2d              INRECH\n' % INRECH) # 
            fid.write('CONSTANT %14g   RECH (PERIOD %d) \n' % (rch_rate, per_i))
        fid.close()


    # -- .nam file with full paths
    fil_nam_0 = MODtest_dir0 + slashstr + fil_nam
    fid = open(fil_nam_0, 'wt')
    fid.write("LIST          7 %s \n" % (MODtest_dir0 + slashstr + 'test.lst')) # MODFLOW output file
    fid.write("BAS6          8 %s \n" % fil_ba6_0)
    fid.write("LPF          11 %s \n" % fil_lpf_0)
    if (fl_recharge):
        fid.write('RCH          18 %s \n' % (MODtest_dir0 + slashstr + fil_rch))

    fid.write('PCG          19 %s \n' % (MODtest_dir0 + slashstr + fil_pcg))
    fid.write('OC           22 %s \n' % (MODtest_dir0 + slashstr + fil_oc))
    fid.write("DIS          10 %s \n" % fil_dis_0)
    fid.write("LMT6         66 %s \n" % (MODtest_dir0 + slashstr + fil_lmt))
    fid.write("DATA(BINARY) 34 %s \n" % (MODtest_dir0 + slashstr + 'test.bud'))  # MODFLOW output file
    fid.write("DATA(BINARY) 51 %s \n" % (MODtest_dir0 + slashstr + 'testhead.dat'))  # MODFLOW output file
    fid.write("DATA         52 %s \n"  % (MODtest_dir0 + slashstr + 'ibound.dat'))  # MODFLOW output file

    fid.close()

    fil_lmt_0 = MODtest_dir0 + slashstr + fil_lmt
    fid = open(fil_lmt_0, 'wt')
    fid.write("# Link-MT3DMS file for MODFLOW-2000, generated by PMWIN \n")
    fid.write("OUTPUT_FILE_NAME   %s\n" % (MODtest_dir0 + slashstr + 'test.flo')) # MODFLOW output file
    fid.write("OUTPUT_FILE_UNIT   333 \n")
    fid.write("OUTPUT_FILE_HEADER Standard \n")
    fid.write("OUTPUT_FILE_FORMAT Unformatted \n")
    fid.close()


    # -- Print instructions to run model:
    print(f"MODFLOW input files created in {MODtest_dir0}")




In [25]:
write_new_files = False
if write_new_files:
    #homogenous case
    write_modflow_input_files('homogenous_k') 
    #top layer K = 9, bottom = 3
    write_modflow_input_files('2_layer_9_3', Ks = {'0': (9, 25), '1': (3, 25)})
    #top layer = 3, bottom = 9
    write_modflow_input_files('2_layer_3_9', Ks = {'0': (3, 25), '1': (9, 25)})
    #'avg' K case
    write_modflow_input_files('avgd_k', Ks = {'0': (9, 25), '1': (3, 25)}, layer_averaged_K = True) 

MODFLOW input files created in C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs/homogenous_k
MODFLOW input files created in C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs/2_layer_9_3
MODFLOW input files created in C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs/2_layer_3_9


### Running simulations 
The subprocess module in python allows for command line executions to be run via jupyter

In [3]:
def run_mf_sim(model_dir): 
    cmd = ['mf2005', f'{model_dir}/test.nam'] #pass specific simulation to command
    result = subprocess.run(cmd, stdout = subprocess.PIPE) #run command 
    return(result.stdout,  result.stderr)

In [41]:
run_simulations = False
if run_simulations: 
    run_mf_sim('./model_runs/2_layer_3_9')
    run_mf_sim('./model_runs/2_layer_9_3')
    run_mf_sim('./model_runs/homogenous_k')
    run_mf_sim('./model_runs/avgd_K')


(b'\r\n                                  MODFLOW-2005     \r\n    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL\r\n                             Version 1.12.00 2/3/2017                        \r\n\r\n Using NAME file: ./model_runs/homogenous_k/test.nam \r\n Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/21 12:20:11\r\n\r\n\r\n WARNING READING LMT PACKAGE INPUT DATA:\r\n [STANDARD] HEADER NO LONGER SUPPORTED; [EXTENDED] HEADER USED INSTEAD.\r\n Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.\r\n Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/21 12:20:11\r\n Elapsed run time:  0.006 Seconds\r\n\r\n  Normal termination of simulation\r\n',
 None)

### 2D Case

In [11]:
def write_modflow_input_files_2D(input_dir, **params):
    '''I just refactored the existing code to be a function 
    where input parameters can be passed to adjust the initial conditions 
    and parameters of the model. '''
    MODtest_dir = input_dir #dictionary where input files are stored
    MOD_dir = 'C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs'
    os.makedirs('C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs', exist_ok = True) #ensure dictionary exists
    slashstr = '/'
    #set parameters to passed values or default values 
    #model geometry
    nlay = params.get('nlay', 50)
    ncol = params.get('ncol', 1)
    domain_len = params.get('domain_len', 0.3)
    domain_bot_elev = params.get('domain_bot_elev', -1)
    domain_top_elev = params.get('domain_top_elev', 0)
    Z = (domain_top_elev - domain_bot_elev)
    dz = Z/nlay

    #head
    TopHead = params.get('TopHead', 1.5)
    left_head = params.get("left_head", 1.5)
    right_head = params.get("right_head", 1.1)
    dh = right_head - left_head
    dh_dx = dh / domain_len
    vertgrad = params.get('vertgrad', -0.0038) # dh/dz, <0 for upward flux
    fl_recharge = params.get('fl_recharge', 0)
    rch_rate= params.get('rch_rate', 5e-4)

    #K
    Ks = params.get('Ks', {'0' : (6, 50)}) #a dictionary of layer number and (K, thickness) where 0 = top and increases going down
    layer_averaged_K = params.get('layer_averaged_K', False) #if True, compute averaged K given the K 
    hydcond = np.ones((nlay, ncol))  
    def calculate_avg_K(Ks, dz):#Ks = directory of conductivity layers as described above, dz = cell length
        layer_start = 0
        total_thickness = 0
        L_K = 0
        for layer, (K, thickness) in Ks.items():
            layer_thickness = thickness * dz
            L_K += layer_thickness/K
            total_thickness += layer_thickness 
        K_avg = total_thickness/L_K
        return K_avg
    if layer_averaged_K:
        Kavg = calculate_avg_K(Ks, dz)
        Ks = {'0': (Kavg, nlay)}

    #set hydcond values according to Ks 
    layer_start = 0
    for layer, (K, thickness) in Ks.items():
        layer_end = layer_start + thickness 
        hydcond[layer_start:layer_end] =K
        layer_start = layer_end 

    # -- everything below this point is the same as the original script! 
    # -- names of input files (created with this script)
    fil_ba6 = 'test_1D.ba6'
    fil_dis = 'test_1D.dis'
    fil_lpf = 'test_1D.lpf'
    fil_nam = 'test.nam'
    fil_lmt = 'test.lmt'

    # the following always have pre-set entries
    fil_pcg = 'test.pcg'
    fil_oc = 'test.oc'

    # the following is created only if recharge stress package is used
    if (fl_recharge):
        fil_rch = 'test.rch'   
    MODtest_dir0 = MOD_dir + slashstr + MODtest_dir

    if not (os.path.exists(MODtest_dir0)):
        os.makedirs(MODtest_dir0)

    # LPF parameters
    flow_filunit = 34  # 0: no cell-by-cell flow terms written, 
                        # >0: cell-by-cell flow terms written to this unit number when "SAVE BUDGET", 
                        # <0: cell-by-cell flow for constant-head cells in listing file when "SAVE BUDGET"
                        
    # - discretization
    DELR = domain_len / ncol  
    DELC = DELR
    top_bound = 1 if fl_recharge else -1
    IBOUND = np.ones((nlay, ncol))  # 1: variable head
    IBOUND[:,0] = -1  # -1: const head at left
    IBOUND[:,-1] = -1  # -1: const head at right
    IBOUND[0,:] = top_bound  # -1: const head at top
    IBOUND[-1,0] = 0  # -1: no flow at bottom 
    init_head_x = left_head + dh_dx * DELC * np.arange(0, ncol, 1)
    initHead = np.tile(init_head_x, (nlay, 1))
    if ((fl_recharge) and np.size(np.where(IBOUND[0,:] < 0) > np.array(0))):
        print('Error! Top layer must be variable head (IBOUND > 0) to receive recharge! Exiting...\n')


    # get layer elevations
    botm = np.linspace(domain_bot_elev, domain_top_elev, nlay+1).reshape(nlay+1, -1)
    botm = botm[-2::-1]
    top_sys = domain_top_elev  # if head > top, this is confined condition


    # -- pcg and oc files are not changed with this script
    fil_pcg_0 = MODtest_dir0 + slashstr + fil_pcg
    fid = open(fil_pcg_0, 'wt')
    fid.write("# Preconditioned conjugate-gradient package\n")
    fid.write("        50        30         1      MXITER, ITER1, NPCOND\n")
    fid.write("  0000.001      .001        1.         2         1         1      1.00\n")
    fid.write(" HCLOSE,      RCLOSE,    RELAX,    NBPOL,     IPRPCG,   MUTPCG    damp\n")
    fid.close()

    fil_oc_0 = MODtest_dir0 + slashstr + fil_oc
    fid = open(fil_oc_0, 'wt')
    fid.write("HEAD PRINT FORMAT 20\n")
    fid.write("HEAD SAVE UNIT 51\n")
    fid.write("COMPACT BUDGET AUX\n")
    fid.write("IBOUND SAVE UNIT 52\n")
    fid.write("PERIOD 1 STEP 1\n")
    fid.write("   PRINT HEAD\n")
    fid.write("   SAVE HEAD\n")
    fid.write("   PRINT BUDGET\n")
    fid.write("   SAVE BUDGET\n")
    fid.write("   SAVE IBOUND\n")
    fid.close()

    #-- ba6 file
    fil_ba6_0 = MODtest_dir0 + slashstr + fil_ba6


    fid = open(fil_ba6_0, 'wt')
    fid.write("# basic package file --- x-section %d layers, %d column\n" % (nlay, ncol))
    fid.write("XSECTION\n")
    fid.write("INTERNAL          1 (FREE)  3\n")
    for i in range(nlay):
        for j in range(ncol):
            fid.write("%4d " % np.transpose(IBOUND)[j, i])
        fid.write("\n")
    fid.write("    999.99  HNOFLO\n")
    fid.write("INTERNAL          1.0 (FREE)  3\n")
    for i in range(nlay):
        for j in range(ncol):
            fid.write("%7g " % np.transpose(initHead)[j, i])
        fid.write("\n")
    fid.close()

    # -- dis file
    # for now, assume these inputs:
    nrow = 1  # 1 for x-section
    nper = 1  # number stress periods
    itmuni = 4  # time units, 4: day
    lenuni = 2  # length units, 2: m
    LAYCBD = np.zeros((1, nlay))  # 0: no quasi-3D confining bed under layer
    perlen = 1  # length of period (doesn't matter for steady-state)
    nstp = 1  # number of time steps (doesn't matter for steady-state)
    tsmult = 1  # multiplier for length of successive time steps
    SsTr_flag = 'SS'  # 'SS' for steady-state, 'Tr' for transient

    fil_dis_0 = MODtest_dir0 + slashstr + fil_dis


    fid = open(fil_dis_0, 'wt')
    fid.write(" %d  %d  %d  %d  %d  %d " % (nlay, nrow, ncol, nper, itmuni, lenuni))
    fid.write("  NLAY,NROW,NCOL,NPER,ITMUNI,LENUNI\n")
    for i in range(nlay):
        fid.write(" %d" % LAYCBD[0, i])
    fid.write("\n")

    fid.write("CONSTANT %14g   DELR\n" % DELR)
    fid.write("CONSTANT %14g   DELC\n" % DELC)
    fid.write("CONSTANT %14g   TOP of system\n" % top_sys)
    for ii in range(1, nlay+1):
        a = botm[ii-1]
        fid.write("CONSTANT %14g   Layer BOTM layer %4d\n" % (a[0], ii))
    for ii in range(1, nper+1):
        fid.write(" %g %d %g %s        PERLEN, NSTP, TSMULT, Ss/Tr (stress period %4d)\n" % (perlen, nstp, tsmult, SsTr_flag, ii))
    fid.close()


    # -- top of lpf file (use Ksat_realization or other to create Ksat inputs)
    # assumed input values
    hdry = 1e30  # head assigned to dry cells
    nplpf = 0    # number of LPF parameters (if >0, key words would follow)
    laytyp = np.zeros((nlay, 1)) 
    laytyp[0, 0] = 1  # top is "covertible", rest are "confined"
    layave = np.zeros((nlay, 1))  # harmonic mean for interblock transmissivity
    chani = np.ones((nlay, 1))   # constant horiz anisotropy mult factor (for each layer)
    layvka = np.zeros((nlay, 1))  # flag 0: vka is vert K >0 is vertK/horK ratio
    laywet = np.zeros((nlay, 1))  # flag 0: wetting off



    fil_lpf_0 = MODtest_dir0 + slashstr +  fil_lpf
    fid = open(fil_lpf_0, 'wt')
    fid.write("# LPF package inputs\n")
    fid.write("%d %g %d    ILPFCB,HDRY,NPLPF\n" % (flow_filunit, hdry, nplpf))
    for i in range(nlay):
        fid.write("%2d" % laytyp[i, 0])
    fid.write("     LAYTYP\n")
    for i in range(nlay):
        fid.write("%2d" % layave[i, 0])
    fid.write("     LAYAVE\n")
    for i in range(nlay):
        fid.write("%2d" % chani[i, 0])
    fid.write("     CHANI\n")
    for i in range(nlay):
        fid.write("%2d" % layvka[i, 0])
    fid.write("     LAYVKA\n")
    for i in range(nlay):
        fid.write("%2d" % laywet[i, 0])
    fid.write("     LAYWET\n")


    # -- Write HKSAT in .lpf file
    for lay in range(1, nlay+1):
        fid.write("INTERNAL   1.000E-00 (FREE)    0            HY layer  %d\n" % (lay))
        for j in range(ncol):    
            fid.write(" %4.1f" % hydcond[lay-1, j])
        fid.write(" \n")

        fid.write("INTERNAL   1.000E-00 (FREE)    0            VKA layer  %d\n" % (lay))
        for j in range(ncol):    
            fid.write(" %4.1f" % hydcond[lay-1, j])
        fid.write(" \n")
    fid.close()


    # -- .rch file
    if (fl_recharge):
        NRCHOP = 3  # option 3: apply recharge rate to highest active cell in column
        IRCHBD = -1 # <0 to skip writing cell-by-cell flow terms to file
        INRECH = 1 # >0: read in recharge values below (<0: use recharge from previous stress period)
        fil_rch_0 = MODtest_dir0 + slashstr + fil_rch
        fid = open(fil_rch_0, 'wt')
        fid.write('# recharge package \n')  
        fid.write('    %2d    %2d        NRCHOP,IRCHBD\n' % (NRCHOP, IRCHBD)) 
        for per_i in range(1, nper+1):
            fid.write('    %2d              INRECH\n' % INRECH) # 
            fid.write('CONSTANT %14g   RECH (PERIOD %d) \n' % (rch_rate, per_i))
        fid.close()


    # -- .nam file with full paths
    fil_nam_0 = MODtest_dir0 + slashstr + fil_nam
    fid = open(fil_nam_0, 'wt')
    fid.write("LIST          7 %s \n" % (MODtest_dir0 + slashstr + 'test.lst')) # MODFLOW output file
    fid.write("BAS6          8 %s \n" % fil_ba6_0)
    fid.write("LPF          11 %s \n" % fil_lpf_0)
    if (fl_recharge):
        fid.write('RCH          18 %s \n' % (MODtest_dir0 + slashstr + fil_rch))

    fid.write('PCG          19 %s \n' % (MODtest_dir0 + slashstr + fil_pcg))
    fid.write('OC           22 %s \n' % (MODtest_dir0 + slashstr + fil_oc))
    fid.write("DIS          10 %s \n" % fil_dis_0)
    fid.write("LMT6         66 %s \n" % (MODtest_dir0 + slashstr + fil_lmt))
    fid.write("DATA(BINARY) 34 %s \n" % (MODtest_dir0 + slashstr + 'test.bud'))  # MODFLOW output file
    fid.write("DATA(BINARY) 51 %s \n" % (MODtest_dir0 + slashstr + 'testhead.dat'))  # MODFLOW output file
    fid.write("DATA         52 %s \n"  % (MODtest_dir0 + slashstr + 'ibound.dat'))  # MODFLOW output file

    fid.close()

    fil_lmt_0 = MODtest_dir0 + slashstr + fil_lmt
    fid = open(fil_lmt_0, 'wt')
    fid.write("# Link-MT3DMS file for MODFLOW-2000, generated by PMWIN \n")
    fid.write("OUTPUT_FILE_NAME   %s\n" % (MODtest_dir0 + slashstr + 'test.flo')) # MODFLOW output file
    fid.write("OUTPUT_FILE_UNIT   333 \n")
    fid.write("OUTPUT_FILE_HEADER Standard \n")
    fid.write("OUTPUT_FILE_FORMAT Unformatted \n")
    fid.close()

    # -- Print instructions to run model:
    print(f"MODFLOW input files created in {MODtest_dir0}")

In [9]:
write_modflow_input_files_2D('2d_flow', ncol = 100, nlay = 50, domain_len = 100, left_head = 1.5, right_head = 1.1)
run_mf_sim('./model_runs/2d_flow')

MODFLOW input files created in C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs/2d_flow


(b'\r\n                                  MODFLOW-2005     \r\n    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL\r\n                             Version 1.12.00 2/3/2017                        \r\n\r\n Using NAME file: ./model_runs/2d_flow/test.nam \r\n Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/25 12:03:22\r\n\r\n\r\n WARNING READING LMT PACKAGE INPUT DATA:\r\n [STANDARD] HEADER NO LONGER SUPPORTED; [EXTENDED] HEADER USED INSTEAD.\r\n Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.\r\n Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/25 12:03:23\r\n Elapsed run time:  0.174 Seconds\r\n\r\n  Normal termination of simulation\r\n',
 None)

In [12]:
#recharge 
write_modflow_input_files_2D('2d_flow_recharge', ncol = 100, nlay = 50, domain_len = 100, left_head = 1.5, right_head = 1.1, fl_recharge = 1)
run_mf_sim('./model_runs/2d_flow_recharge')

MODFLOW input files created in C:/Users/huan1428/Documents/esci5331_hydro_modeling/assignment_5/model_runs/2d_flow_recharge


(b'\r\n                                  MODFLOW-2005     \r\n    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL\r\n                             Version 1.12.00 2/3/2017                        \r\n\r\n Using NAME file: ./model_runs/2d_flow_recharge/test.nam \r\n Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/25 12:04:56\r\n\r\n\r\n WARNING READING LMT PACKAGE INPUT DATA:\r\n [STANDARD] HEADER NO LONGER SUPPORTED; [EXTENDED] HEADER USED INSTEAD.\r\n Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.\r\n Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/25 12:04:56\r\n Elapsed run time:  0.067 Seconds\r\n\r\n  Normal termination of simulation\r\n',
 None)

### Graphing model outputs

In [7]:
init_head_x = [100, 101, 102, 103]
nlay= 10


array([[100, 101, 102, 103],
       [100, 101, 102, 103],
       [100, 101, 102, 103],
       [100, 101, 102, 103],
       [100, 101, 102, 103],
       [100, 101, 102, 103],
       [100, 101, 102, 103],
       [100, 101, 102, 103],
       [100, 101, 102, 103],
       [100, 101, 102, 103]])